In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load image
image = cv2.imread('Data_histEq/Stage 2/FHPL_5214046596_20220815113721_20220829113334_Image_17_0008_2022-08-29_11-36-20-611_1526.png', 0)  # Load image in grayscale

# Initialize the region mask
region_mask = np.zeros_like(image)

# Define parameters
seed_point = (100, 150)  # Seed point chosen for region growing
threshold = 10  # Intensity difference threshold

# Get the intensity value of the seed point
seed_value = image[seed_point]

# Stack to store the pixels to be explored
pixel_stack = [seed_point]

# List of directions to explore neighbors (4-connected neighbors)
neighbors = [(-1, 0), (1, 0), (0, -1), (0, 1)]

# Function for Region Growing
def region_growing(image, region_mask, seed_point, seed_value, threshold):
    while len(pixel_stack) > 0:
        # Get the current pixel from stack
        x, y = pixel_stack.pop()

        # Check bounds
        if x < 0 or x >= image.shape[0] or y < 0 or y >= image.shape[1]:
            continue
        
        # Check if the pixel is already labeled in the region mask
        if region_mask[x, y] == 0:
            # Compute the intensity difference
            if abs(int(image[x, y]) - int(seed_value)) < threshold:
                # Add pixel to the region
                region_mask[x, y] = 255  # Mark it as part of the region

                # Explore neighbors
                for dx, dy in neighbors:
                    new_x, new_y = x + dx, y + dy
                    pixel_stack.append((new_x, new_y))

    return region_mask

# Perform region growing
region_mask = region_growing(image, region_mask, seed_point, seed_value, threshold)

# Display the result
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title("Original Image")
plt.subplot(1, 2, 2)
plt.imshow(region_mask, cmap='gray')
plt.title("Region Grown Segmentation")
plt.show()


In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# Load the image
image = cv2.imread('Data_histEq/Stage 2/FHPL_5214046596_20220815113721_20220829113334_Image_17_0008_2022-08-29_11-36-20-611_1526.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply a binary threshold to the image
_, binary_image = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Remove noise using morphological opening
kernel = np.ones((3, 3), np.uint8)
opening = cv2.morphologyEx(binary_image, cv2.MORPH_OPEN, kernel, iterations=2)

# Perform dilation to obtain sure background
sure_bg = cv2.dilate(opening, kernel, iterations=3)

# Distance transform to obtain sure foreground (regions likely to be objects)
dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
_, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)

# Identify the unknown region (subtract sure foreground from sure background)
sure_fg = np.uint8(sure_fg)
unknown = cv2.subtract(sure_bg, sure_fg)

# Marker labeling: Label the sure foreground with distinct integers
_, markers = cv2.connectedComponents(sure_fg)

# Add one to all markers so that sure background is not 0
markers = markers + 1

# Mark the region of unknown with zero
markers[unknown == 255] = 0

# Apply the Watershed algorithm
markers = cv2.watershed(image, markers)

# Mark boundaries in red (set the boundary pixels to -1)
image[markers == -1] = [0, 0, 255]

# Display results
plt.figure(figsize=(10, 5))
plt.subplot(1, 3, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Watershed Segmentation')
plt.subplot(1, 3, 2)
plt.imshow(binary_image, cmap='gray')
plt.title('Binary Image')
plt.subplot(1, 3, 3)
plt.imshow(dist_transform, cmap='gray')
plt.title('Distance Transform')
plt.show()


In [ ]:
import cv2
from PIL import Image
import numpy as np
import os
from skimage.filters import gabor
from skimage import img_as_float
import warnings
warnings.filterwarnings('ignore')

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

def enhance_edges(path):
    image = cv2.imread(path)

    # Check if the image was loaded properly
    if image is None:
        print(f"Error: Image not found or unable to load: {path}")
        return None

    prewitt_x = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.float32)
    prewitt_y = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]], dtype=np.float32)
    
    # Apply Prewitt operator
    edge_x = cv2.filter2D(image, -1, prewitt_x)
    edge_y = cv2.filter2D(image, -1, prewitt_y)
    
    # Combine edges
    edges = np.uint8(np.sqrt(edge_x**2 + edge_y**2))
    edges = cv2.cvtColor(edges, cv2.COLOR_BGR2GRAY)

    image = cv2.normalize(image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    image = np.uint8(image)

    if len(image.shape) == 3:
        image[:,:,1] = clahe.apply(image[:,:,1])
        image[:,:,0] = clahe.apply(image[:,:,0])
        image[:,:,2] = clahe.apply(image[:,:,2])

    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image_float = img_as_float(image)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 50))

    top_hat = cv2.morphologyEx(image, cv2.MORPH_TOPHAT, kernel)
    bottom_hat = cv2.morphologyEx(image, cv2.MORPH_BLACKHAT, kernel)

    enhanced_image = cv2.add(image, top_hat)
    enhanced_image = cv2.subtract(enhanced_image, bottom_hat)

    return enhanced_image

def gab(img_path):
    img = enhance_edges(img_path)
    
    if img is None:  # Check if image enhancement failed
        return None

    filt_real, filt_imaginary = gabor(img, 1/4, 30, sigma_x=1, sigma_y=1)
    img = Image.fromarray(filt_real)
    
    return img

data_dir = "Data/FVR"
save_dir_base = "Enhanced"

for images in os.listdir(data_dir):
    image_path = os.path.join(data_dir, images)
    enhanced_image = gab(image_path)
    
    if enhanced_image is None:  # Skip saving if image is None
        continue
    
    dest_path = os.path.join(save_dir_base, os.path.relpath(image_path, data_dir))
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)  # Ensure directory exists
    enhanced_image.save(dest_path)


In [ ]:
# import os

# def remove_spaces_from_filenames(root_dir):
#     # Walk through all subdirectories and files
#     for dirpath, dirnames, filenames in os.walk(root_dir):
#         # Loop through all the files
#         for filename in filenames:
#             # If the filename contains spaces
#             if 'img' in filename:
#                 # Create the old and new file paths
#                 old_file_path = os.path.join(dirpath, filename)
#                 new_filename = filename.replace('img', '')  # Replace spaces with underscores
#                 new_file_path = os.path.join(dirpath, new_filename)
                
#                 # Rename the file
#                 os.rename(old_file_path, new_file_path)
#                 print(f"Renamed: {old_file_path} -> {new_file_path}")

# # Replace 'root_folder' with the path to the directory containing the folders and files
# root_folder = 'Data'
# remove_spaces_from_filenames(root_folder)


In [ ]:
from PIL import Image
import os

# Path to the folder containing .tif files
input_folder = r"C:\Users\xerom\Downloads\mask_BV\mask"
output_folder = r"C:\Users\xerom\Downloads\mask_BV\masks"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through all .tif files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".tif") or file_name.endswith(".tiff"):
        # Open the .tif image
        tif_path = os.path.join(input_folder, file_name)
        img = Image.open(tif_path)
        
        # Convert to .png and save
        png_path = os.path.join(output_folder, file_name.replace(".tif", ".png").replace(".tiff", ".png"))
        img.save(png_path)

        print(f"Converted {file_name} to {os.path.basename(png_path)}")


In [ ]:
from PIL import Image
import os

# Path to the folder containing .jpg files
input_folder = r"C:\Users\xerom\Downloads\Image and Masks Blood Vessels (Stage 2)\Image and Masks Blood Vessels (Stage 2)\Masks"
output_folder = r"C:\Users\xerom\Downloads\Image and Masks Blood Vessels (Stage 2)\Image and Masks Blood Vessels (Stage 2)\Mask"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Loop through all .jpg files in the input folder
for file_name in os.listdir(input_folder):
    if file_name.endswith(".jpg") or file_name.endswith(".jpeg"):
        # Open the .jpg image
        jpg_path = os.path.join(input_folder, file_name)
        img = Image.open(jpg_path)

        # Convert to .png and save
        png_path = os.path.join(output_folder, file_name.replace(".jpg", ".png").replace(".jpeg", ".png"))
        img.save(png_path)

        print(f"Converted {file_name} to {os.path.basename(png_path)}")


In [ ]:
import numpy as np

# Example 3x3 confusion matrix
confusion_matrix = np.array([
    [75, 4, 2],  # True class A
    [0, 69, 3],  # True class B
    [1, 7, 75]   # True class C
])

# Total accuracy
accuracy = np.trace(confusion_matrix) / np.sum(confusion_matrix)

# Precision, Recall, and F1 Score for each class
precision = []
recall = []
f1_score = []

for i in range(len(confusion_matrix)):
    tp = confusion_matrix[i, i]
    fp = np.sum(confusion_matrix[:, i]) - tp
    fn = np.sum(confusion_matrix[i, :]) - tp
    
    class_precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    class_recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    class_f1 = 2 * class_precision * class_recall / (class_precision + class_recall) if (class_precision + class_recall) > 0 else 0
    
    precision.append(class_precision)
    recall.append(class_recall)
    f1_score.append(class_f1)

# Macro-average (unweighted average)
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = np.mean(f1_score)

# Weighted-average (weighted by the number of true instances per class)
weights = np.sum(confusion_matrix, axis=1) / np.sum(confusion_matrix)
weighted_precision = np.sum(precision * weights)
weighted_recall = np.sum(recall * weights)
weighted_f1 = np.sum(f1_score * weights)

# Print results
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro Precision: {macro_precision:.4f}")
print(f"Macro Recall: {macro_recall:.4f}")
print(f"Macro F1 Score: {macro_f1:.4f}")
print(f"Weighted Precision: {weighted_precision:.4f}")
print(f"Weighted Recall: {weighted_recall:.4f}")
print(f"Weighted F1 Score: {weighted_f1:.4f}")


Accuracy: 0.93
Macro Precision: 0.93
Macro Recall: 0.93
Macro F1 Score: 0.93
Weighted Precision: 0.93
Weighted Recall: 0.93
Weighted F1 Score: 0.93
